# Intro

The code below use brainstorm3 data, stored in local files.

It calls the best-plugin through the MatLab Engine.

274 Sensors
361 values per sensor

In [1]:
# Start MATLAB engine
import matlab.engine
eng = matlab.engine.start_matlab("-desktop")

In [2]:
# Add paths to MATLAB
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/MATLAB/best-brainstorm/best'))", nargout=0)
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/MATLAB/best-brainstorm/minFunc'))", nargout=0)
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/dev/python-brainstorm/nirstorm-master/bst_plugin'))", nargout=0)
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/dev/brainstorm3'))", nargout=0)

In [3]:
# Load the options and head model
eng.eval("MEMOptions = importdata('./MEMOptions.breakpoint.mat')", nargout=0)
eng.eval("load('./OPTIONS.mat')", nargout=0)
eng.eval("load('./HeadModel.mat')", nargout=0)
eng.eval("VertConn = importdata('./VertConn.mat').VertConn", nargout=0)

In [4]:
# Change the gain matrix to a struct
eng.eval("""
    temp_struct = struct('matrix', HeadModel.Gain, 'modality', 'MEG');
    HeadModel.Gain = repmat(temp_struct, 1, 1);
""", nargout=0)

# Add the vertex connectivity to the head model
eng.eval("""
    HeadModel.vertex_connectivity = VertConn
""", nargout=0)

# Only use the first n time points (for testing, to speed up computation)
eng.eval("""
    MEMOptions.mandatory.DataTime = MEMOptions.mandatory.DataTime(1:40);
""", nargout=0)

In [5]:
# Call the main MATLAB function
result_struct = eng.eval("be_main_call_return_struct(HeadModel, MEMOptions)", nargout=0)

# Call the MATLAB function
# result_struct = eng.feval(
#     "be_main_call_return_struct",
#     "HeadModel",
#     "MEMOptions",
#     nargout=1)


In [6]:
variables = eng.eval('who', nargout=1)
print(variables) # ['HeadModel', 'MEMOptions', 'OPTIONS', 'Results', 'VertConn', 'ans', 'temp_struct']

ImageGridAmp = eng.eval("Results.ImageGridAmp", nargout=1)

import numpy as np
ImageGridAmp_np = np.array(ImageGridAmp)
print(ImageGridAmp_np.shape)

['HeadModel', 'MEMOptions', 'OPTIONS', 'Results', 'VertConn', 'ans', 'temp_struct']
(17005, 40)


In [7]:
# reading vertices from the head model
vertices = eng.eval("double(full(VertConn))", nargout=1)

# Transform the vertices to a numpy array
vertices_np = np.array(vertices)

In [8]:
# Transform the adjacency matrix to a list of connected vertices (not sure about this)
connected_vertices = np.where(np.sum(vertices_np, axis=1) > 0)[0]

In [11]:
# Create a estimate volume (not sure about this, is it volume ?)
import mne

data = ImageGridAmp_np
connected_vertices_list = [connected_vertices.tolist()]
tmin = 0
tstep = 1
stc = mne.VolSourceEstimate(data, vertices=connected_vertices_list, tmin=tmin, tstep=tstep)

In [12]:
# Show 3d brain with source estimates
source_surface_file = "C:/Users/Ilian/Documents/MATLAB/brainstorm/TutorialIntroduction2/anat/Subject01/tess_cortex_pialcereb_low.mat"

# Load mat file
import scipy.io
mat = scipy.io.loadmat(source_surface_file)

# vertices = mat['Vertices']  # Assurez-vous que c'est la bonne clé
faces = mat['Faces'] - 1  # MATLAB indexe à 1, Python à 0

# Créer un 'source space' pour un hémisphère, simulant un hémisphère gauche
src_data = {
    'vertno': np.arange(vertices_np.shape[0]),  # indices des vertices
    'rr': vertices_np,  # positions des vertices (doivent être en mètres)
    'nn': mat.get('VertNormals', np.zeros(vertices_np.shape)),  # Normales aux vertices
    'nuse': vertices_np.shape[0],  # nombre de vertices utilisés
    'inuse': np.ones(vertices_np.shape[0], int),  # tous les vertices sont utilisés
    'ntri': faces.shape[0],  # nombre de triangles
    'tris': faces,  # les triangles
    'type': 'surf',  # Spécification du type d'espace source
    'coord_frame': mne.io.constants.FIFF.FIFFV_COORD_MRI,
    'id': 101  # ID fictif pour l'hémisphère gauche
}

src = mne.SourceSpaces([src_data], dict(dist=None))  # Simplifié, normalement plus d'infos

In [ ]:
%pip install nibabel
%pip install nilearn

In [14]:
from mne.viz import plot_volume_source_estimates
print(type(stc))
plot_volume_source_estimates(src_data, stc)

<class 'mne.source_estimate.VolSourceEstimate'>


TypeError: stc must be an instance of VolSourceEstimate, got <class 'dict'> instead.